In [1]:
# !pip install kaggle pandas sqlalchemy psycopg2-binary yfinance pandas_datareader requests beautifulsoup4 matplotlib Pillow flask_cors

In [2]:
import os
import zipfile
import pandas as pd
import requests
import psycopg2
from bs4 import BeautifulSoup
from io import StringIO
from sqlalchemy import create_engine
from psycopg2 import sql
import shutil
import stat
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
# Set the Kaggle configuration directory to a custom path
custom_kaggle_path = '.kaggle'
os.environ['KAGGLE_CONFIG_DIR'] = custom_kaggle_path

# Define paths
zip_file_path = 'Resources/sandp500.zip'
extract_path = 'Resources/sandp500'

# Function to handle permission errors
def handle_remove_readonly(func, path, exc_info):
    os.chmod(path, stat.S_IWRITE)
    func(path)

# Remove existing ZIP file if it exists
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)

# Remove existing folder and its contents if it exists
if os.path.exists(extract_path):
    shutil.rmtree(extract_path, onerror=handle_remove_readonly)

# Download the S&P 500 dataset from Kaggle and store in the Resources folder
os.system('kaggle datasets download -d camnugent/sandp500 -p Resources')

# Unzip the downloaded file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Load dataset into DataFrame
csv_file_path = os.path.join(extract_path, 'all_stocks_5yr.csv')
df = pd.read_csv(csv_file_path)

# Delete the ZIP file and extracted folder after loading the dataset
os.remove(zip_file_path)
shutil.rmtree(extract_path, onerror=handle_remove_readonly)

# Remove rows with missing values
cleaned_df = df.dropna()

# Convert date column to datetime using .loc
cleaned_df.loc[:, 'date'] = pd.to_datetime(cleaned_df['date'], errors='coerce')

# Drop rows with invalid dates (NaT values)
cleaned_df = cleaned_df.dropna(subset=['date'])

# Rename columns for clarity using .loc
cleaned_df = cleaned_df.rename(columns={'Name': 'ticker', 'date': 'date', 'open': 'open_price', 
                                        'close': 'close_price', 'low': 'low_price', 'high_price': 'high_price', 
                                        'volume': 'volume'})

# Convert date column back to datetime to ensure correct format for PostgreSQL
cleaned_df.loc[:, 'date'] = pd.to_datetime(cleaned_df['date'], format='%Y-%m-%d')

# Display df info
cleaned_df.info()
print('==================================================================================')

# Fetch S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Request page content with headers to avoid being blocked
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table', {'class': 'wikitable'}) 
    
    if len(tables) > 0:
        table = tables[0]
        
        # Wrap the table HTML string in a StringIO object
        table_html = StringIO(str(table))
        
        # Read the table directly into a DataFrame using pandas
        ticker_df = pd.read_html(table_html)[0]
        
        # Select only the Ticker Symbol and Company Name columns
        ticker_df = ticker_df[['Symbol', 'Security']]
        
        # Rename columns for clarity
        ticker_df.columns = ['ticker', 'company_name']
        
        # Check for duplicates and drop them
        ticker_df = ticker_df.drop_duplicates()

        # Check for missing/null values and drop rows with any missing values
        ticker_df = ticker_df.dropna()

        # Verify that all tickers in cleaned_df are present in ticker_df
        missing_tickers = cleaned_df[~cleaned_df['ticker'].isin(ticker_df['ticker'])]['ticker'].unique()
        if len(missing_tickers) > 0:
            print(f"Missing tickers in ticker_tb: {missing_tickers}")

        # Display DataFrame info after cleaning
        ticker_df.info()
    else:
        print("Error: S&P 500 company table not found on Wikipedia.")
else:
    print(f"Error: Failed to fetch page, status code {response.status_code}")
print('==================================================================================')

# Define initial and target database parameters
initial_db_params = {
    'dbname': 'postgres',  
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

db_params = {
    'dbname': 'stocks_dashboard_db',  
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

# Functions for database operations
def terminate_sessions(cursor, dbname):
    cursor.execute(sql.SQL("""
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = %s AND pid <> pg_backend_pid();
    """), [dbname])

def drop_database(cursor, dbname):
    cursor.execute("SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = %s AND pid <> pg_backend_pid();", [dbname])
    cursor.execute(sql.SQL("DROP DATABASE IF EXISTS {}").format(sql.Identifier(dbname)))

def create_database(cursor, dbname):
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))

def execute_sql_file(cursor, sql_file_path):
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()
    cursor.execute(sql.SQL(sql_commands))

sql_file_path = 'stocks_dashboard_db_schema.sql'

# Connect to the initial database and create the target database
try:
    # Connect to the initial database (postgres)
    connection = psycopg2.connect(**initial_db_params)
    connection.autocommit = True
    cursor = connection.cursor()

    # Drop the target database if it exists
    drop_database(cursor, db_params['dbname'])
    print(f"Database {db_params['dbname']} dropped successfully.")

    # Create the target database
    create_database(cursor, db_params['dbname'])
    print(f"Database {db_params['dbname']} created successfully.")

    # Close the initial connection
    cursor.close()
    connection.close()

    # Connect to the newly created target database (stocks_dashboard_db)
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Execute the SQL schema file to set up the database
    execute_sql_file(cursor, sql_file_path)
    connection.commit()
    print("SQL schema file executed successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

# Upload data from DataFrames to PostgreSQL
def upload_df_to_table(connection_string, table_name, df):
    engine = create_engine(connection_string)
    df.to_sql(table_name, engine, if_exists='append', index=False)
    return len(df)

# Ensure column names in cleaned_df and ticker_df match the table schema
ticker_df.columns = ['ticker', 'company_name']
cleaned_df.columns = ['date', 'open_price', 'close_price', 'low_price', 'high_price', 'volume', 'ticker']

# Upload the DataFrames to PostgreSQL tables
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

try:
    ticker_rows = upload_df_to_table(connection_string, 'ticker_tb', ticker_df)
    print(f"Data for ticker_tb uploaded successfully. Total rows: {ticker_rows}")
    
    sp500_rows = upload_df_to_table(connection_string, 'sp500_tb', cleaned_df)
    print(f"Data for sp500_tb uploaded successfully. Total rows: {sp500_rows}")

except Exception as e:
    print(f"An error occurred: {e}")

# Add fake data to portfolio_tb
# Create a DataFrame with specific ticker symbols
portfolio_data = {
    'ticker': ['AAPL', 'NVDA', 'AMZN', 'TSLA', 'NFLX', 'MCD'],
    'shares': [50, 30, 40, 25, 35, 45]
}

# Create a DataFrame
portfolio_df = pd.DataFrame(portfolio_data)

# Upload the DataFrame to PostgreSQL
try:
    engine = create_engine(connection_string)
    portfolio_df.to_sql('portfolio_tb', engine, if_exists='append', index=False)
    print(f"Data for portfolio_tb uploaded successfully. Total rows: {len(portfolio_df)}")
except Exception as e:
    print(f"An error occurred: {e}")


<class 'pandas.core.frame.DataFrame'>
Index: 619029 entries, 0 to 619039
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         619029 non-null  object 
 1   open_price   619029 non-null  float64
 2   high         619029 non-null  float64
 3   low_price    619029 non-null  float64
 4   close_price  619029 non-null  float64
 5   volume       619029 non-null  int64  
 6   ticker       619029 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 37.8+ MB
Missing tickers in ticker_tb: ['AAL' 'AAP' 'ABC' 'ADS' 'AET' 'AGN' 'AIV' 'ALK' 'ALXN' 'AMG' 'ANDV'
 'ANTM' 'APC' 'ARNC' 'ATVI' 'AYI' 'BBT' 'BHF' 'BHGE' 'BLL' 'CA' 'CBG'
 'CBS' 'CELG' 'CERN' 'CHK' 'CMA' 'COG' 'COL' 'COTY' 'CSRA' 'CTL' 'CTXS'
 'CXO' 'DISCA' 'DISCK' 'DISH' 'DPS' 'DRE' 'DWDP' 'DXC' 'ESRX' 'ETFC'
 'EVHC' 'FBHS' 'FB' 'FISV' 'FLIR' 'FLR' 'FLS' 'FL' 'FTI' 'GGP' 'GPS' 'GT'
 'HBI' 'HCN' 'HCP' 'HOG' 'HP' 'HRB' 'HRS' 'ILMN' 'INFO' 'JEC'

In [4]:
result = cleaned_df.groupby('ticker').agg(
    distinct_date_count=('date', 'nunique'),
    min_date=('date', 'min'),
    max_date=('date', 'max')
).reset_index()

print(result)

    ticker  distinct_date_count             min_date             max_date
0        A                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
1      AAL                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
2      AAP                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
3     AAPL                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
4     ABBV                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
..     ...                  ...                  ...                  ...
500    XYL                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
501    YUM                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
502    ZBH                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
503   ZION                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00
504    ZTS                 1259  2013-02-08 00:00:00  2018-02-07 00:00:00

[505 rows x 4 columns]


In [5]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.graph_objects as go  


# # Set the style for the plots
# sns.set(style="darkgrid")

# # Function to plot historical price trends, volatility, moving averages, etc., for a given ticker
# def plot_stock_analysis(ticker):
#     # Load stock data for the selected ticker
#     stock_data = cleaned_df[cleaned_df['ticker'] == ticker]
    
#     if stock_data.empty:
#         print(f"No data found for ticker: {ticker}")
#         return

#     # Ensure the data is sorted by date
#     stock_data = stock_data.sort_values(by='date')
    
#     # Calculate daily returns
#     stock_data['daily_return'] = stock_data['close_price'].pct_change()

#     # Historical Price Trends show the overall price movement.
#     plt.figure(figsize=(10, 6))
#     plt.plot(stock_data['date'], stock_data['close_price'], label='Closing Price', color='b', alpha=0.7)
#     plt.title(f"Historical Price Trends for {ticker}")
#     plt.xlabel('Date')
#     plt.ylabel('Price (USD)')
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

#     # RSI (Relative Strength Index) shows the stock’s potential overbought/oversold condition.
#     def compute_rsi(data, window=14):
#         delta = data.diff()
#         gain = delta.where(delta > 0, 0)
#         loss = -delta.where(delta < 0, 0)
#         avg_gain = gain.rolling(window=window, min_periods=1).mean()
#         avg_loss = loss.rolling(window=window, min_periods=1).mean()
#         rs = avg_gain / avg_loss
#         rsi = 100 - (100 / (1 + rs))
#         return rsi

#     stock_data['RSI'] = compute_rsi(stock_data['close_price'])

#     plt.figure(figsize=(10, 6))
#     plt.plot(stock_data['date'], stock_data['RSI'], label='RSI', color='purple')
#     plt.axhline(30, color='red', linestyle='--')
#     plt.axhline(70, color='green', linestyle='--')
#     plt.title(f"RSI for {ticker}")
#     plt.xlabel('Date')
#     plt.ylabel('RSI')
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

#     # Bollinger Bands provide a measure of volatility and potential overbought/oversold signals.
#     rolling_mean = stock_data['close_price'].rolling(window=20).mean()
#     rolling_std = stock_data['close_price'].rolling(window=20).std()
#     stock_data['Bollinger_Upper'] = rolling_mean + (rolling_std * 2)
#     stock_data['Bollinger_Lower'] = rolling_mean - (rolling_std * 2)

#     plt.figure(figsize=(10, 6))
#     plt.plot(stock_data['date'], stock_data['close_price'], label='Close Price', color='blue', alpha=0.7)
#     plt.plot(stock_data['date'], stock_data['Bollinger_Upper'], label='Upper Band', color='red', linestyle='--')
#     plt.plot(stock_data['date'], stock_data['Bollinger_Lower'], label='Lower Band', color='red', linestyle='--')
#     plt.title(f"Bollinger Bands for {ticker}")
#     plt.xlabel('Date')
#     plt.ylabel('Price (USD)')
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.show()
   

#     # Drawdown Chart Using cumulative returns for drawdown
#     stock_data['Cumulative_Returns'] = (1 + stock_data['daily_return']).cumprod()
#     stock_data['Drawdown'] = stock_data['Cumulative_Returns'] - stock_data['Cumulative_Returns'].cummax()
    
#     plt.figure(figsize=(10, 6))
#     plt.fill_between(stock_data['date'], stock_data['Drawdown'], color='red', alpha=0.6)
#     plt.title(f"Drawdown for {ticker}")
#     plt.xlabel('Date')
#     plt.ylabel('Drawdown')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()
 

# # Example usage: Allow the user to input/select the ticker
# ticker_input = input("Enter a ticker symbol (e.g., AAPL, MSFT, NFXL): ").upper()  # Convert input to uppercase
# plot_stock_analysis(ticker_input)


Historical Price Trends (Closing Price)
What it shows: This plot shows how the stock’s closing price has changed over time.
Why it’s useful: The historical price trend helps investors to visualize the long-term price movement of a stock. It can indicate trends, peaks, and dips over a period of time.
How it's calculated: The plot simply takes the Close_Price of the stock and plots it against the Date. This is the most basic way to track the stock's performance over time.

Relative Strength Index (RSI)
What it shows: The RSI is a momentum oscillator that measures the speed and change of price movements. It helps determine whether a stock is overbought or oversold.
Why it’s useful: An RSI above 70 typically indicates that the stock is overbought (potentially overvalued), and an RSI below 30 suggests that the stock is oversold (potentially undervalued). It is used to identify potential buy and sell signals.
How it's calculated: The RSI is calculated using the average gain and loss over a 14-day period. It is plotted on a scale of 0 to 100. If the RSI is above 70, the stock is considered overbought; if it’s below 30, the stock is considered oversold.

Bollinger Bands
What it shows: Bollinger Bands are volatility bands placed above and below a moving average. The distance between the bands increases or decreases based on volatility.
Why it’s useful: The upper and lower bands provide insights into the overbought or oversold conditions of the stock. When the price reaches the upper band, the stock may be considered overbought, and when it reaches the lower band, it may be considered oversold.
How it's calculated:
The Middle Band is the 20-period simple moving average (SMA) of the closing prices.
The Upper Band is the 20-period SMA plus two times the standard deviation of the closing prices.
The Lower Band is the 20-period SMA minus two times the standard deviation of the closing prices.